In [1]:
import pandas as pd
import numpy as np

In [2]:
DATA_PATH_ORIG = r'../data/capture20110812.pcap.netflow.labeled.csv'
DATA_PATH_NEW = r'../data/scenario_3.csv'

In [3]:
column_names = ['_', '_2', 'duration', 'protocol', 'src_addr', '->', 'dst_addr', 'flags', 'tos', 'packets', 'bytes', 'flows', 'label']

df = pd.read_csv(
    DATA_PATH_ORIG,
    delimiter='\s+',
    skiprows=[0],
    header=None, 
    names=column_names,
    index_col=False,
    parse_dates={"start_date": [0,1]}
)
df = df.sort_values('start_date')
df.head()

,start_date,duration,protocol,src_addr,->,dst_addr,flags,tos,packets,bytes,flows,label
0,2011-08-12 15:24:01.105,4.677,TCP,74.125.108.243:80,->,147.32.86.187:55707,PA_,0,452,682688,1,Background
1,2011-08-12 15:24:01.105,3.110,TCP,147.32.84.164:22,->,83.208.193.123:15173,PA_,0,5,506,1,LEGITIMATE
2,2011-08-12 15:24:01.105,4.678,TCP,147.32.86.187:55707,->,74.125.108.243:80,A_,0,106,6510,1,LEGITIMATE
3,2011-08-12 15:24:01.106,4.989,TCP,217.168.211.184:3953,->,147.32.84.14:22,PA_,0,1393,95812,1,Background
4,2011-08-12 15:24:01.113,4.988,UDP,173.9.132.155:54369,->,147.32.84.59:15046,INT,0,811,680898,1,Background


In [4]:
portsrc = df["src_addr"].str.split(":", n = 1, expand = True) 
df['src_port'] = portsrc[1]
df["src_addr"] = portsrc[0]

portdest = df["dst_addr"].str.split(":", n = 1, expand = True) 
df['dst_port'] = portdest[1]
df["dst_addr"] = portdest[0]

df['src_port']=pd.to_numeric(df['src_port'], errors='coerce').fillna(-1).astype(np.int64)
df['dst_port']=pd.to_numeric(df['dst_port'], errors='coerce').fillna(-1).astype(np.int64)

In [5]:
df.drop(columns = ['->'], inplace=True)

In [6]:
df.head()

,start_date,duration,protocol,src_addr,dst_addr,flags,tos,packets,bytes,flows,label,src_port,dst_port
0,2011-08-12 15:24:01.105,4.677,TCP,74.125.108.243,147.32.86.187,PA_,0,452,682688,1,Background,80,55707
1,2011-08-12 15:24:01.105,3.110,TCP,147.32.84.164,83.208.193.123,PA_,0,5,506,1,LEGITIMATE,22,15173
2,2011-08-12 15:24:01.105,4.678,TCP,147.32.86.187,74.125.108.243,A_,0,106,6510,1,LEGITIMATE,55707,80
3,2011-08-12 15:24:01.106,4.989,TCP,217.168.211.184,147.32.84.14,PA_,0,1393,95812,1,Background,3953,22
4,2011-08-12 15:24:01.113,4.988,UDP,173.9.132.155,147.32.84.59,INT,0,811,680898,1,Background,54369,15046


In [7]:
df.isnull().values.any()

False

In [8]:
df.to_csv(DATA_PATH_NEW, index=False)